# Demo: Reproducible Features in CrocoDash

## Set up a custom regional CESM-MOM6 run using reproducible features

A typical workflow of utilizing CrocoDash consists of four main steps:

1. Generate a regional MOM6 domain.
2. Create the CESM case.
3. Prepare ocean forcing data.
4. Build and run the case.

This notebook demonstrates key reproducibility features in CrocoDash, including interactive case assembly, tides configuration, and `user_nl_mom` block management, which can further streamline this workflow.

All steps are tracked in the provenance files: `README.case`, `case_history.json`, `object_histories.json`, and `user_nl_history.json` for full reproducibility.

# SECTION 1: Assemble a Case from History

Use the interactive widget to rebuild a `Case` from global history.

You can select `Grid`, `Topo`, `VGrid`, and a forcing configuration from previous runs.

Compatibility checks between objects (e.g., `Grid` and `Topo` attributes) are performed automatically under the hood, so only valid combinations are shown in the UI.

## Provenance and reproducibility:

* Every selection and instantiation is logged in `case_history.json` and `object_histories.json`.
* The `README.case` file is automatically updated with a summary of the configuration and workflow code.
* All files and settings used are tracked for future restoration or auditing.


After you select the `Grid`, `Topo`, `VGrid`, and forcing configuration, the widget pauses and presents you with a choice:

*Generate Forcings*: This option will immediately configure and process the atmospheric and oceanic forcings for your case, updating all relevant configuration and provenance files.

*Leave Case As-Is*: This option allows you to build the case without generating forcings, so you can manually configure them later or use further customized workflows.

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


*The Case Assembler only works if at least one Case has been previously instantiated. Ensure that this has been previously done.*

In [15]:
from CrocoDash.assemble_case import CaseAssembler

# Launch the widget (edit paths as needed)
CaseAssembler.assemble_case(
    caseroot="/glade/u/home/akwong/croc_cases",
    inputdir="/glade/u/home/akwong/croc_input",
    cesmroot="/glade/work/akwong/cesm_installs/CESM"
)

HTML(value='<h1>Case Assembler</h1>')

Output(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', border_right='1px solid gra…

Output()

# SECTION 2: Configure and Process Tides


After assembling your case, you can configure tidal constituents and process tides for reproducibility.

These actions update both the runtime configuration `user_nl_mom` and the provenance files `README.case`, `case_history.json`, `user_nl_history.json`.

In [ ]:
# Example: configure tides for the case
case.configure_tides(
    tidal_constituents=["M2"],
    tpxo_elevation_filepath="/glade/derecho/scratch/manishrv/ice/croc/tpxo/h_tpxo9.v1.nc",
    tpxo_velocity_filepath="/glade/derecho/scratch/manishrv/ice/croc/tpxo/u_tpxo9.v1.nc"
)

# Process tides (generates boundary files and updates user_nl_mom)
case.process_tides()

# SECTION 3: Remove and Append Tides in `user_nl_mom`

You can remove the tides block from `user_nl_mom` and clear related config fields:

In [ ]:
case.remove_user_nl_block("Tides", clear_config=True)

To (re-)append a custom tides block:

In [ ]:
case.append_user_nl_block(
    "Tides",
    [
        ("TIDES", "True"),
        ("TIDE_M2", "True"),
        ("CD_TIDES", 0.0018),
        ("TIDE_USE_EQ_PHASE", "True"),
        ("TIDE_REF_DATE", "1850, 1, 1"),
        ("OBC_TIDE_ADD_EQ_PHASE", "True"),
        ("OBC_TIDE_N_CONSTITUENTS", 1),
        ("OBC_TIDE_CONSTITUENTS", '"M2"'),
        ("OBC_TIDE_REF_DATE", "1850, 1, 1"),
    ],
    tpxo_elevation_filepath="/glade/derecho/scratch/manishrv/ice/croc/tpxo/h_tpxo9.v1.nc",
    tpxo_velocity_filepath="/glade/derecho/scratch/manishrv/ice/croc/tpxo/u_tpxo9.v1.nc"
)

# SECTION 4: Undo/Redo `user_nl_mom` Changes

You can undo or redo changes to `user_nl_mom` for full reproducibility:

In [ ]:
from visualCaseGen.custom_widget_types.case_tools import undo_user_nl, redo_user_nl

# Undo the last user_nl change
undo_user_nl(case.caseroot, do_exec=True, case_obj=case)

# Redo the last undone user_nl change
redo_user_nl(case.caseroot, do_exec=True, case_obj=case)

This is useful for exploring different configurations or reverting mistakes.

*Tip:*
All changes to `user_nl_mom` blocks and tides configuration are logged and can be traced in the case history files for full provenance. Read more on the documentation regarding reproducibility in CrocoDash here: _____TO BE MADE DOCS_____